# ClipCard Simulation — DevOps / SRE


# ClipCard Notebook Plan (auto-generated 2025-10-13)

This notebook is a scaffold for the **ClipCard (Risk & Recheck)** seedpack.

- **Base schema** (`schema/clipcard.schema.v1.json`) is required.
- **Extended schema** (`schema/clipcard.schema.v1e.json`) is optional.
- Works with CSVs under `data/clipmap/*.csv` and any `**/*.clipcard.json` in your repo.

## Inputs
- `schema/clipcard.schema.v1.json` (required)
- `schema/clipcard.schema.v1e.json` (optional)
- `data/clipmap/cards.csv` and `data/clipmap/events.csv` (optional; created by CI)
- Any `**/*.clipcard.json` examples you add

## Outputs
- Console metrics and/or CSVs in `notebooks/_out/`
- Plots (if matplotlib installed)
- Notes you can copy into Field Reports

> Optional installs (inside a notebook cell):
>
> ```
> %pip install pandas jsonschema matplotlib
> ```



## What this sim does

- Estimates how often ClipCard triggers on high-risk work
- Sketches reduction in **false approvals** when ClipCard is used on those items
- Lets you tweak parameters to match your environment

**This is a toy model**, useful to reason about thresholds before a real field trial.


In [ ]:

# Parameters (edit as needed)
n_items = 10_000
p_high_impact = 0.25         # share with impact≥4 (1–5 scale)
p_high_uncertainty = 0.30    # share with uncertainty≥4
p_rev_ge4 = 0.10             # reversibility≥4
p_coup_ge4 = 0.12            # coupling≥4

false_approval_rate_high = 0.15   # baseline (no ClipCard) on high-risk
clipcard_effect = 0.60            # multiplicative factor (0.60 = 40% reduction)

import random
random.seed(42)

def rand1to5(p_hi):
    x = random.randint(1,5)
    if random.random() < p_hi and x < 5:
        x += 1
    return x

def triggers():
    impact = rand1to5(p_high_impact)
    uncertainty = rand1to5(p_high_uncertainty)
    iu = impact * uncertainty >= 18
    rev = random.random() < p_rev_ge4
    coup = random.random() < p_coup_ge4
    ext = rev or coup
    return iu or ext

trig_count = 0
base_false = 0
with_clip_false = 0

for _ in range(n_items):
    if triggers():
        trig_count += 1
        base_false += (1 if random.random() < false_approval_rate_high else 0)
        with_clip_false += (1 if random.random() < (false_approval_rate_high * clipcard_effect) else 0)

share_trig = trig_count / n_items if n_items else 0.0
benefit = base_false - with_clip_false
print(f"Items: {n_items}")
print(f"Triggered: {trig_count} ({share_trig:.1%})")
print(f"False approvals (base): {base_false}")
print(f"False approvals (with ClipCard): {with_clip_false}")
print(f"Benefit (avoided false approvals): {benefit} (~{(benefit/max(base_false,1))*100:.1f}% of base false approvals)")
